# Analyzing EDI files set using Pandas 

From EDI measurment csv file (edi_measurement.csv) to plottings

## Fei Zhang
### 2017-09

### See Similar Notebooks:
* geopandas_csv2shp


In [ ]:
import os, sys,glob
%matplotlib inline
from IPython.display import display

import folium
from shapely.geometry import mapping

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [ ]:
def csv_to_geopdf(csvfile):
    
    pdf = pd.read_csv(csvfile)
    mt_locations = [Point(xy) for xy in zip(pdf.LON, pdf.LAT)]
    #OR pdf['geometry'] = pdf.apply(lambda z: Point(z.lon, z.lat), axis=1)
    #if you want to df = df.drop(['Lon', 'Lat'], axis=1)
    crs = {'init': 'epsg:4326'}  # WGS84
    crs = {'init': 'epsg:4283'}  # GDA94
    geo_df = gpd.GeoDataFrame(pdf, crs=crs, geometry=mt_locations)

    return geo_df

In [ ]:
# path2csv='E:/Data/MT_Datasets/WenPingJiang_SHP/PhaseTensorTipper_Params_1.0986Hz.csv'
#path2csv='E:/example_data_edi_files/edi_measurement.csv'
#path2csv='E:/tmp4/edi_measurement.csv'


from mtpy.core.edi_collection import EdiCollection

#Supress default INFO logging
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
logger.propagate = False

EDI_FOLDER='E:/Data/MT_Datasets/GA_UA_edited_10s-10000s'
EDI_FOLDER='E:/Data/MT_Datasets/3D_MT_data_edited_fromDuanJM'

edi_list= glob.glob(EDI_FOLDER + '/*.edi')

ediob= EdiCollection(edi_list)

path2csv = ediob.create_measurement_csv(dest_dir='E:/tmp7')                                   

print(path2csv)
                                        
path2dir=os.path.dirname(path2csv)
mygdf=csv_to_geopdf(path2csv)

In [ ]:
# mygdf.head()
mygdf.shape

In [ ]:
mygdf.iloc[0]     # [-1] last row or feature

In [ ]:
mygdf.crs

In [ ]:
shape_fname=path2csv.upper().replace('.CSV', '.SHP')   # 'geopandas_made.shp'
outshp=os.path.join(path2dir,shape_fname)


In [ ]:
outshp

In [ ]:
mygdf.to_file(outshp, driver='ESRI Shapefile')

In [ ]:
import matplotlib as mpl

mpl.rcParams['lines.linewidth'] = 2
# mpl.rcParams['lines.color'] = 'r'

#mpl.rcParams['figure.figsize']=(8,30)

In [ ]:
#gdf_utm54=mygdf.to_crs({'init':'epsg:32754'})  #.plot()

#outshp=os.path.join(path2dir,'geopandas_made_utm54.shp')

#gdf_utm54.to_file(outshp, driver='ESRI Shapefile')


In [ ]:

geojson = mygdf.to_json()

# output jeojson?
# geojson big-string

bounds = mygdf.total_bounds


In [ ]:
print bounds

In [ ]:
mygdf.columns

In [ ]:
mygdf['ZXYre'].describe()

In [ ]:
# unique stations
station_list= mygdf['STATION'].unique()

In [ ]:
# plot a station's MT data
stid=8

mystation=station_list[stid] #'pb27'

#mygdf2 = mygdf.where (mygdf['FREQ'] == 0.000122)  # not working
mygdf2 = mygdf.loc[mygdf['STATION'] == mystation]


In [ ]:
mygdf2.shape

In [ ]:
mydf2=pd.DataFrame(mygdf2)
mydf2['PERIOD'] = mydf2.apply(lambda z: 1.0/z.FREQ, axis=1)

In [ ]:
mydf2s= mydf2.sort_values('PERIOD', axis=0, ascending=True, inplace=False)

In [ ]:
ax=mydf2s.plot(x='PERIOD', y=['RHOxy', 'RHOyx'], logx=True, logy=True, marker='*', 
            title="Station=%s"%mystation)

ax.set_ylabel('App. Res. ($\mathbf{\Omega \cdot m}$)')
ax.set_xlabel('Period (s)')

In [ ]:
ax=mydf2s.plot(x='PERIOD', y=['PHSxy', 'PHSyx'], logx=True, logy=False, marker='^'
            ,title="Station=%s"%mystation)
ax.set_ylabel('Phase (Deg)')
ax.set_xlabel('Period (s)')

In [ ]:
180*74.36

In [ ]:
80*73.97 +100*74.68

In [ ]:
FIN